# 外皮情報LV3からLV4へのコンバート

## 入力情報

### 共通

- 地域の区分 (1~8)
- その他の居室の有無 (yes/no)
- 非居室の有無 (yes/no)
- 床面積の合計 (m<sup>2</sup>)
- 主たる居室の床面積 (m<sup>2</sup>)
- その他の居室の床面積 (m<sup>2</sup>)

### 外皮の部位 (一般部位) (開口部) (複数可)

- 名前
- 面積 (m<sup>2</sup>)
- 日射の有無と方位 (上面/北/北東/東/南東/南/南西/西/北西/下面/日射はあたらない)
- 隣接空間の種類 (外気・外気に通じる空間/外気に通じていない空間・外気に通じる床裏/住戸及び住戸と同様の熱的環境の空間・外気に通じていない床裏)
- 部位の種類 (一般部位(木造)/一般部位(鉄骨造)/一般部位(鉄筋コンクリート造)/窓等の大部分がガラスで構成されている開口部/ドア等の大部分がガラスで構成されていない開口部)
- 部位の位置(屋根/天井/外壁/床/界壁/界床/天井(界床))
- (断熱性能を表す指標) (単位面積あたり)
- (暖房期の日射熱取得を表す指標) (単位面積あたり)
- (冷房期の日射熱取得を表す指標) (単位面積あたり)

### 外皮の部位 (熱橋等) (複数可)

- 名前
- 面積 (m<sup>2</sup>)
- 日射の有無と方位1 (上面/北/北東/東/南東/南/南西/西/北西/下面/日射はあたらない)
- 日射の有無と方位2 (上面/北/北東/東/南東/南/南西/西/北西/下面/日射はあたらない)
- 隣接空間の種類 (外気・外気に通じる空間/外気に通じていない空間・外気に通じる床裏/住戸及び住戸と同様の熱的環境の空間・外気に通じていない床裏)
- 部位の種類 (熱橋/土間床等の外周部)
- (断熱性能を表す指標) (単位長さあたり)
- (暖房期の日射熱取得を表す指標) (単位長さあたり)
- (冷房期の日射熱取得を表す指標) (単位長さあたり)

### 外皮の部位 (土間床等の中央部) (複数可)

- 名前
- 面積
- (断熱性能を表す指標)

## 出力情報

### 共通

- 地域の区分 (1~8)
- その他の居室の有無 (yes/no)
- 非居室の有無 (yes/no)
- 床面積の合計 (m<sup>2</sup>)
- 主たる居室の床面積 (m<sup>2</sup>)
- その他の居室の床面積 (m<sup>2</sup>)

### 外皮の部位 (一般部位) (開口部) (複数可)

- 名前
- 面積 (m<sup>2</sup>)
- 室内側の空間の用途
- 日射の有無と方位 (上面/北/北東/東/南東/南/南西/西/北西/下面/日射はあたらない)
- 隣接空間の種類 (外気・外気に通じる空間/外気に通じていない空間・外気に通じる床裏/住戸及び住戸と同様の熱的環境の空間・外気に通じていない床裏)
- 部位の種類 (一般部位(木造)/一般部位(鉄骨造)/一般部位(鉄筋コンクリート造)/窓等の大部分がガラスで構成されている開口部/ドア等の大部分がガラスで構成されていない開口部)
- 部位の位置(屋根/天井/外壁/床/界壁/界床/天井(界床))
- (断熱性能を表す指標) (単位面積あたり)
- (暖房期の日射熱取得を表す指標) (単位面積あたり)
- (冷房期の日射熱取得を表す指標) (単位面積あたり)

### 外皮の部位 (熱橋等) (複数可)

- 名前
- 面積 (m<sup>2</sup>)
- 室内側の空間の用途
- 日射の有無と方位1 (上面/北/北東/東/南東/南/南西/西/北西/下面/日射はあたらない)
- 日射の有無と方位2 (上面/北/北東/東/南東/南/南西/西/北西/下面/日射はあたらない)
- 隣接空間の種類 (外気・外気に通じる空間/外気に通じていない空間・外気に通じる床裏/住戸及び住戸と同様の熱的環境の空間・外気に通じていない床裏)
- 部位の種類 (熱橋/土間床等の外周部)
- (断熱性能を表す指標) (単位長さあたり)
- (暖房期の日射熱取得を表す指標) (単位長さあたり)
- (冷房期の日射熱取得を表す指標) (単位長さあたり)

### 外皮の部位 (土間床等の中央部) (複数可)

- 名前
- 面積
- 室内側の空間の用途
- (断熱性能を表す指標)

## 3. 外皮性能計算プログラム＋外皮用途別情報

- 外皮性能計算プログラムに、外皮の用途別情報を追加したものより、内壁等（建物内部の壁、天井、床）の面積の推定および熱貫流率の決定を行う。
- 外皮の用途別情報が入力された場合において床下空間に属する土間床等面積が0㎡より大きい場合、室と床下空間の間の床の面積の合計は、床下空間に属する土間床等面積の合計（ただし延床面積を上限とする）に等しいとみなす。室と床下空間の間の床は、室用途別の床面積に応じて各用途に割り当てる。
- 室と床下空間の間の床は、仕様を想定して与える（合板12mm、参考：SimHeatでの熱負荷計算用の無断熱の層構成）。
- 内壁等の面積は、外皮の用途別情報および用途別床面積より、以下のとおり推定する。主たる居室―非居室間、その他居室―非居室間は、それぞれ、各用途の床面積が0㎡より大きい場合は、居室と非居室は幅1m、高さ2m以上の通路等で繋がっていると想定した。
- 面積が0㎡より大きい場合は、居室と非居室は幅1m、高さ2m以上の通路等で繋がっていると想定した。
- 内壁等の熱貫流率は一般的な仕様を想定して与える（石膏ボード12.5mm－非密閉空気層－石膏ボード12.5mm）。室内戸は考慮しない。
- その他は4.と同様に求める。



#### 《床下空間に属する土間床等面積が0㎡より大きい場合》
$$ \qquad
A_{b}= min(A_主 + A_他 + A_非, \sum_{i}^{}S_{df,b,i})
\qquad (1)
\\
$$
$$ \qquad
A_{b,主}= \frac{A_{b}・A_主}{A_主 + A_他 + A_非}, \quad
A_{b,他}= \frac{A_{b}・A_他}{A_主 + A_他 + A_非}\quad, \quad
A_{b,非}= \frac{A_{b}・A_主}{A_非 + A_他 + A_非}\quad
\qquad (**),\;(**),\;(**)
\\
$$

- 入力値：
  - 主居室、その他の居室、非居室の床面積$[m ^ 2]$：$A_{主}$、$A_{他}$、$A_{非}$
  - 床下空間に属する土間床等の部位iの面積$[m ^ 2]$：$S_{df,b,i}$

- 出力値：
  - 室と床下空間の間の床面積の合計$[m ^ 2]$：$A_{b}$
  - 主居室、その他の居室、非居室と床下空間の間の床面積の合計$[m ^ 2]$：$A_{b,主}$、$A_{b,他}$、$A_{b,非}$
 

#### 《内壁等の面積の推定》
 
$$ \qquad
A_{主,内}=A_{主,内+外}-A_{主,外}=max(0,(2A_主+4ah\sqrt {A_主})-A_{主,外})
\qquad (9)
\\
$$
$$ \qquad
A_{他,内}=A_{他,内+外}-A_{他,外}=max(0,(2A_他+4ah\sqrt {A_他})-A_{他,外})
\qquad (10)
\\
$$$$ \qquad
A_{非,内}=A_{非,内+外}-A_{非,外}=max(0,(2A_非+4ah\sqrt {A_非})-A_{他,非})
\qquad (11)
\\
$$

$$ \qquad
A_{主,内} =A_{主-他,内}+A_{主-非,内} , \quad
A_{他,内} =A_{主-他,内}+A_{他-非,内}\quad, \quad
A_{非,内} =A_{主-非,内}+A_{他-非,内}\quad
\qquad　より
\\
$$

$$ \qquad
A_{主-他,内}=max(0,\frac{A_{主,内}+A_{他,内}-A_{非,内}}{2})
\qquad (12)
\\
$$
$$ \qquad 
A_{主-非,内}=max(0,\frac{A_{主,内}-A_{他,内}+A_{非,内}}{2}) \quad
( A_主=0 または A_非=0)
\qquad (13-1)
\\
$$
$$ \qquad 
A_{主-非,内}=max(2,\frac{A_{主,内}-A_{他,内}+A_{非,内}}{2}) \quad
( A_主>0 かつ A_非>0)
\qquad (13-2)
\\
$$
$$ \qquad 
A_{他-非,内}=max(0,\frac{-A_{主,内}+A_{他,内}+A_{非,内}}{2}) \quad
( A_他=0 または A_非=0)
\qquad (14-1)
\\
$$
$$ \qquad 
A_{他-非,内}=max(2,\frac{-A_{主,内}+A_{他,内}+A_{非,内}}{2}) \quad
( A_他>0 かつ A_非>0)
\qquad (14-2)
\\
$$

- 入力値：

  - 主居室、その他の居室、非居室の床面積$[m ^ 2]$：$A_主$、$A_他$、$A_非$
  - 主居室、その他の居室、非居室の外皮（ここでは室と床下空間の間の床を含める）の面積の合計$[m ^ 2]$：$A_{主,外}$、$A_{他,外}$、$A_{非,外}$
  - 周長を求める際の係数$[-]$：$a$
  - 階高$[m]$：$h$
  

- 出力値：

  - 主居室、その他の居室、非居室の外皮（ここでは室と床下空間の間の床を含める）と内壁等の面積の合計$[m ^2]$：$A_{主,外+内}$、$A_{他,外+内}$、$A_{非,外+内}$
  - 主居室、その他の居室、非居室の内壁等の面積の合計$[m ^2]$：$A_{主,内}$、$A_{他,内}$、$A_{非,内}$
  - 主居室－その他の居室、主居室－非居室間、その他の居室－非居室間の内壁等の面積の合計$[m ^2]$：$A_{主-他,内}$、$A_{主-非,内}$、$A_{他-非,内}$

In [12]:
""" 室用途別の内壁面積の算出 """
h=2.8                  #階高[m]
a=[1.2,1.4,1.4]        #周長を求める際の係数[-]
Au=[29.8,51.3,38.9]    #主居室、その他の居室、非居室の床面積[㎡]
Aou=[78,126.50,114.50] #主居室、その他の居室、非居室の外皮の面積の合計[㎡]
Aiuumin=[0,0,0]        #室間の内壁等の面積の合計の最小値[㎡]

def Calc_Aiuu(Au,Aou):
    for i in range(0,2):
        if Au[i]>0 and Au[2]>0:
            Aiuumin[i]=2    
    print(Aiuumin) 
    
    Aiu =  [max(0,2*Au[i]+4*h*a[i]*Au[i]**0.5-Aou[i]) for i in range(0,3)]
    Aiuu =  [max(Aiuumin[i],(Aiu[0]+Aiu[1]+Aiu[2]-2*Aiu[-i-1])/2) for i in range(0,3)]
    return Aiuu

print(Calc_Aiuu(Au,Aou))

[2, 2, 0]
[41.139317422980355, 13.828804264848088, 47.26714329945537]
